In [ ]:
import json
import numpy as np

from flask import Flask, request

app = Flask(__name__)


@app.route('/rank', methods =['POST'])
def ranking():
    j = request.json
    inverted_index = j['data']
    # list with relevant docID
    documents = j['documents']
    # list with tokens from the query
    words = j['words']
    # weight for the term in the title
    w = 5
    
    def tf(document):
        # len(document['pos']) - number of term appearances in the text
        # len(document['pos_title']) - number of term appearances in the title
        return 1 + np.log((len(document['pos']) + w*len(document['pos_title'])) / document['doc_length'])
    
           
    # create tf_idf -> {docID: tf_idf}
    tf_idf = {key: 0 for key in documents}   
    for term in words:  
        idf_for_term = idf[term]
        for doc in inverted_index[term]:
            if doc['docID'] in documents:
                tf_idf[doc['docID']] += tf(doc) * idf_for_term 
            
           

    # sorting documents
    ranked = sorted(tf_idf.items(), key=lambda kv: kv[1], reverse=True)
    ranked = [d[0] for d in ranked]
      
    return json.dumps({'status':'ok', 'ranked': ranked})

@app.route("/rank/idf", methods=['POST'])
def refresh_idf():
    global idf
    j = request.json
    # idf -> {term:idf}
    idf = j['data']
    return json.dumps({'status':'ok'})

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13541)